In [1]:
import numpy as np
import pandas as pd

In [3]:
#机器数量
print '请输入机器数量：'
M=int(raw_input())

请输入机器数量：
4


In [6]:
#执行时间
pro_time={}
df=pd.read_csv('pro_time.csv')
for i in range(0,len(df)):
    pro_time[str(i+1)]=df.loc[i].p

In [7]:
#约束条件—用一个字典表示
cons={}
df=pd.read_csv('file_cons.csv')
for i in range(0, len(df)):
    cons[str(df.loc[i].pri)] = df.loc[i].next.split()

In [8]:
#计算起始点和最终点
number_p=len(pro_time)
all_point=list([str(x) for x in range(1,number_p+1)])
l=cons.keys()
g=[]
start_point=[]
end_point=[]
for i in l:
    g=g+cons[i]
g=list(set(g))
for i in l:
    if i not in g:
        start_point.append(i)
for i in g:
    if i not in l:
        end_point.append(i)

# print start_point
# print end_point



In [9]:
#开始时间，结束时间初始化
start_time={}
end_time={}
for i in all_point:
    start_time[i]=0
    end_time[i]=0
for i in all_point:
    end_time[i]=start_time[i]+pro_time[i]

def update_time():
    for i in cons:
        for j in cons[i]:
            if end_time[i]>start_time[j]:
                start_time[j]=end_time[i]
                end_time[j]=start_time[j]+pro_time[j]
    return 0

In [10]:
def find_all_paths(graph, start, end, path=[]):
    path = path + [start]
    if start == end:
        return [path]
    if not graph.has_key(start):
        return []
    paths = []
    for node in graph[start]:
        if node not in path:
            newpaths = find_all_paths(graph, node, end, path)
            for newpath in newpaths:
                paths.append(newpath)
    return paths

In [11]:
#计算每条路的权重
def weight(path,pro_time):
    if len(path)!=0:
        tot=0
        for i in path:
            tot=tot+pro_time[i]
    else:
        return None
    return tot

In [12]:
#计算每个点的权重
def max_weight(point,end_point):
    max_w=[]
    for i in end_point:
        for k in find_all_paths(cons, point,i, path=[]):
            max_w.append(weight(k,pro_time))
    return max(max_w)

In [13]:
L_weight={}
for i in all_point:
    L_weight[i]=max_weight(i,end_point)

S=pd.Series(L_weight)
Sorted_p=S.sort_values(ascending=False)

list_byweight=Sorted_p.index
print '每个作业流的总权重'
print Sorted_p

每个作业流的总权重
1     30
8     22
7     22
4     22
5     21
9     20
2     20
6     10
3     10
12    10
11    10
10    10
dtype: int64


In [14]:

#机器初始化
Machine=[]
for i in range(M):
    dirc={}
    dirc['num']=i+1
    dirc['work']=[]
    dirc['span_time']=0
    Machine.append(dirc)
Machine_df=pd.DataFrame(Machine)


In [15]:

#将一项作业插入到机器中
def put_into2(work):
    l=list(Machine_df['span_time'])
    k=l.index(min(l))
    a=Machine_df.loc[k,'span_time']
    if start_time[work]>a:
        a=start_time[work]
        Machine_df['work'][k].append('0')
    Machine_df['work'][k].append(work)
    Machine_df.loc[k,'span_time']=a+pro_time[work]
    end_time[work]=Machine_df.loc[k,'span_time']
    update_time()
    return 0


In [16]:

for i in list_byweight:
    put_into2(i)


In [17]:

print '排程结果（0代表机器空闲）：'
print Machine_df

排程结果（0代表机器空闲）：
   num  span_time           work
0    1         32   [1, 5, 0, 3]
1    2         32     [8, 9, 12]
2    3         32     [7, 2, 11]
3    4         41  [4, 0, 6, 10]


In [18]:
print '总完工时间为：'
print max(list(Machine_df['span_time']))

总完工时间为：
41


In [19]:
print '每项作业开始时间：'
for i in all_point:
    print i,':',start_time[i]

每项作业开始时间：
1 : 0
2 : 10
3 : 22
4 : 0
5 : 0
6 : 21
7 : 0
8 : 0
9 : 0
10 : 22
11 : 22
12 : 22


In [20]:
print '每项作业的完成时间：'
for i in all_point:
    print i,':',end_time[i]

每项作业的完成时间：
1 : 10
2 : 22
3 : 32
4 : 12
5 : 21
6 : 31
7 : 12
8 : 12
9 : 22
10 : 41
11 : 32
12 : 32
